Euclidean Travelling Salesman Problem

In [16]:
from math import sqrt
import random
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
import networkx as nx
import time

from io import BytesIO
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.support import ui
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Manual ETSP

In [ ]:
with open('./initializer/test_data_1.txt', 'r') as file:
    n_points = int(file.readline())
    rows = [[float(x) for x in line.split(' ')[:]] for line in file]
    cols = [list(col) for col in zip(*rows)]
    
x_cords = cols[1]
y_cords = cols[2]
points = list(zip(x_cords, y_cords))
list_points = np.array([list(a) for a in zip(x_cords, y_cords)])
name2point = {i: j for i, j in zip(count(start=1, step=1), points)}
point2name = {y: x for x,y in name2point.items()}

Auto ETSP

In [2]:
options = webdriver.ChromeOptions()
binary_yandex_driver_file = './initializer/yandexdriver.exe'
driver = webdriver.Chrome(binary_yandex_driver_file, options=options)

ETSP_step = [str(a) for a in range(1, 10)]
step_number = ETSP_step[0]
ETSP_link = "/login?next=lesson%252F110889%252Fstep%252F"+step_number

driver.get("https://stepik.org"+ETSP_link)
    
my_login = 'some.email@gmail.com'
my_password = '*********'

login_bar = ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Log in")))

id_box = ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "id_login_email")))
id_box.send_keys(my_login)

pass_box = ui.WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "id_login_password")))
pass_box.send_keys(my_password)

driver.find_element_by_xpath("//button[text()='Log in']").click()

time.sleep(10)
fileLink_ETSP = [a.get_attribute('href') for a in driver.find_elements_by_xpath('//*[@id="ember140"]/span/ul/li[1]/a')]
fileLink_ETSP.append([a.get_attribute('href') for a in driver.find_elements_by_xpath('//*[@id="ember140"]/span/ul/li[2]/a')])

In [3]:
resp = urlopen(fileLink_ETSP[0])
txt_data = BytesIO(resp.read())
data = [[float(x) for x in line.decode('UTF-8').split(' ')[:]] for line in txt_data.readlines()]

n_points = int(data[0][0])
points = [x[1:] for x in data][1:]
tuple_points = [tuple(a) for a in points]
name2point = {i: j for i, j in zip(count(start=1, step=1), tuple_points)}
point2name = {y: x for x,y in name2point.items()}

In [4]:
def idist(ip1, ip2):
    global points
    p1 = points[ip1]
    p2 = points[ip2]
    return sqrt((float(p1[0]) - float(p2[0]))**2 + (float(p1[1]) - float(p2[1]))**2)

In [5]:
def euclidian_tsp(cords):
    points = cords
    circuit = list()
    start_vertex = 0
    dark_side = set(range(n_points)) - {start_vertex}

    cycle = []
    cycle.append(start_vertex)

    current_vertex = start_vertex
    while len(dark_side) > 0:
        min_distance = None
        best_v = None
        for v in dark_side:
            if ((min_distance is None) or 
                (min_distance > idist(current_vertex, v))):
                min_distance = idist(current_vertex, v)
                best_v = v
        circuit.append((current_vertex, best_v))
        dark_side.remove(best_v)
        current_vertex = best_v
        cycle.append(current_vertex)

    circuit.append((current_vertex, start_vertex))
    circuit_length = sum(idist(*e) for e in circuit)
    OutputCircuit = [a for (a, b) in circuit]
    OutputCircuit.append(start_vertex)

    return OutputCircuit, circuit_length

In [6]:

def tourCost(route):
    totalDistance =0.0
    size = len(route)
    for index in range(size):
        if index == size-1 :
            point2 = route[0]
        else:
            point2 = route[index+1]
        totalDistance +=  idist(route[index], point2)
    return totalDistance

def swapper(route,i,k):
    r1=route[:i] 
    r2=route[i:k] 
    r2=r2[::-1] 
    r3=route[k:]
    return(r1+r2+r3)

def super_duper():
    route, dist = euclidian_tsp(points)
    for i in range(len(points)):
        for j in range(i,len(points)):
            new_route = swapper(route[:],i,j)
            new_dist=tourCost(new_route)

            if(new_dist<dist):
                dist=new_dist
                route=new_route
    route = [a+1 for a in route]
    return route, new_dist

In [37]:
id_tour = super_duper()[0]
costa_ricca = super_duper()[1]
tour_answer = ' '.join(str(e) for e in id_tour)

In [ ]:
cords_tour = []
for i in id_tour:
    cords_tour.append(name2point[i])
    
edge_list = list(zip(id_tour[:-1], id_tour[1:]))

In [ ]:
%load_ext line_profiler
%lprun -f super_duper super_duper()

In [ ]:
print(id_tour)
G = nx.DiGraph()
G.add_nodes_from(id_tour)
G.add_edges_from(edge_list)
edge_colors = ['black']
pos = nx.spiral_layout(G)

plt.figure(1,figsize=(25,8))
x = []
y = []
for point in points:
    x.append(point[0])
    y.append(point[1])
plt.scatter(x, y)
plt.show()

plt.figure(2,figsize=(25,25))
nx.draw(G, pos, node_size=15, edge_color=edge_colors, edge_cmap=plt.cm.Reds, with_labels = True)

plt.show()

In [ ]:
answer_box = ui.WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.TAG_NAME, "textarea")))
answer_box.click()
answer_box.send_keys(tour_answer)